In [17]:
import os
import json
import requests
import minsearch
from google import genai
from dotenv import load_dotenv

In [18]:
load_dotenv()

True

In [19]:
client = genai.Client(api_key = os.getenv('GEMINI_API_KEY'))

In [20]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
docs_raw = docs_response.json()

In [21]:
documents = []

for course_dicts in docs_raw:
    for doc in course_dicts['documents']:
        doc['course'] = course_dicts['course']
        documents.append(doc)

# Using Vector Search 

In [36]:
from qdrant_client import QdrantClient, models

In [37]:
qd_client = QdrantClient("http://localhost:6333")    

In [38]:
from fastembed import TextEmbedding

In [39]:
EMBEDDING_DIMENSIONALITY = 512

In [40]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [ ]:
collection_name = "llm-zoomcamp"

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY, 
        distance=models.Distance.COSINE  
    )
)

In [46]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword" # exact matching on string metadata fields
)

UpdateResult(operation_id=5, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
points = []

#embedding both the question and answer instead of just answers
for i, doc in enumerate(documents):
    text = doc['question'] + " " + doc['text']
    vector=models.Document(text=text, model=model_handle)
    
    point = models.PointStruct(
        id=i, 
        vector=vector,
        payload=doc
)
    points.append(point)

In [103]:
points[0]

PointStruct(id=0, vector=Document(text="Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with ann

In [72]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [84]:
course = "data-engineering-zoomcamp"
def vector_search(query, course, limit=5):
    results = []
    result = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
    query_filter=models.Filter( # filter by course name
        must=[
            models.FieldCondition(
                key="course",
                match=models.MatchValue(value=course)
            )
        ]
        ),
        limit=limit, 
        with_payload=True 
    )
    for point in result.points:
        results.append(point.payload)
    
    return results

In [96]:
def build_prompt(query, search_answers):
    prompt = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE
    
    QUESTION: {question}
    
    CONTEXT: {context}
    """.strip()
    
    context = ''
    for doc in search_answers:
        context = context + f"section: {doc['section']} \nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt.format(question=query, context=context).strip()

    return prompt

In [97]:
def rag(query, course):
    search_results = vector_search(query, course)
    prompt = build_prompt(query, search_results)
    llm_response = request_llm(prompt)

    return llm_response

In [106]:
query = 'I just discovered the course, can I still join?'
query2 = "how do I run Kafka?"

In [109]:
print(rag(query2, course))

To run Kafka with Java, in the project directory, run: `java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java`.
If you encounter a `kafka.errors.NoBrokersAvailable` error, ensure your Kafka broker Docker container is working by using `docker ps` to confirm. Then, in the Docker Compose YAML file folder, run `docker compose up -d` to start all instances.
If you get a "Module “kafka” not found" error when trying to run producer.py, create a virtual environment and run requirements.txt and the python files in that environment. To create a virtual env and install packages (run only once):
```
python -m venv env
source env/bin/activate
pip install -r ../requirements.txt
```
To activate it (you'll need to run it every time you need the virtual env):
```
source env/bin/activate
```
To deactivate it:
```
deactivate
```
This works on MacOS, Linux and Windows - but for Windows the path is slightly different (it's env/Scripts/activate). Also the virtual en